# Exchanging assignment files manually

After an assignment has been created using `nbgrader generate_assignment`, the instructor must actually release that assignment to students. This page describes how to do that using your institution's existing learning management system, assuming that the students will fetch the assignments from - and submit their assignments to - the learning management system.

If this is not the case and you are using nbgrader in a shared server environment (e.g. JupyterHub), you can do this with an exchange implementation (see :doc:`managing_assignment_files`).

Distributing assignments to students and collecting them can be a logistical nightmare. The previous page discussed the built-in exchange directory, but that is not the only option (and in fact, was added later on).  One can also distribute and collect files by other means, such as though your institution's learning management system.  If you are relying on your institution's learning management system to get the submitted versions of notebooks back from students, ``nbgrader`` has some built-in functionality to make theat easier (putting the files in the right place into the course directory via an importer).

One can also do this fully manually, by sending files around.  This may be useful during the testing phase.

## Releasing assignments

In short, to release an assignment, send the files at ``release/{assignment_id}/*`` to your students.  For example, you might post the files on your course page.

## Submitting assignments

When an assignment is submitted, it needs to be placed in ``submitted/{student_id}/{assignment_id}/*``.  The rest of this page describes the built-in ways to do this, if students upload them to a learning management system and you can download them all at once in an archive.  This is called **collecting** the assignment.


## Collecting assignments

Once the students have submitted their assignments and you have downloaded these assignment files from your institution's learning management system, you can get theses files back into ``nbgrader`` by using the ``nbgrader zip_collect`` sub-command.

### Directory Structure:

### Workflow

### Step 1: Download submission files or archives

For demo purposes we have already created the directories needed by the ``nbgrader zip_collect`` sub-command and placed the downloaded assignment submission files and archive (zip) files in there. For example we have one ``.zip`` file and one ``.ipynb`` file:

In [1]:
%%bash

ls -l downloaded/ps1/archive

total ##
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] jupyter.png
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] notebooks.zip
-rw------- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb


But before we can run the ``nbgrader zip_collect`` sub-command we first need to specify a few config options:

In [2]:
%%file nbgrader_config.py

c = get_config()

# Only set for demo purposes so as to not mess up the other documentation
c.CourseDirectory.submitted_directory = 'submitted_zip'

# Only collect submitted notebooks with valid names
c.ZipCollectApp.strict = True

# Apply this regular expression to the extracted file filename (absolute path)
c.FileNameCollectorPlugin.named_regexp = (
    '.*_(?P<student_id>\w+)_attempt_'
    '(?P<timestamp>[0-9\-]+)_'
    '(?P<file_id>.*)'
)

Overwriting nbgrader_config.py


Setting the ``strict`` flag ``True`` skips any submitted notebooks with invalid names.

By default the ``nbgrader zip_collect`` sub-command uses the ``FileNameCollectorPlugin`` to collect files from the ``extracted_directory``. This is done by sending each filename (**absolute path**) through to the ``FileNameCollectorPlugin``, which in turn applies a named group regular expression (``named_regexp``) to the filename.

The ``FileNameCollectorPlugin`` returns ``None`` if the given file should be skipped or it returns an object that must contain the ``student_id`` and ``file_id`` data, and can optionally contain the ``timestamp``, ``first_name``, ``last_name``, and ``email`` data.

Thus if using the default ``FileNameCollectorPlugin`` you must at least supply the ``student_id`` and ``file_id`` named groups. This plugin assumes all extracted files have the same filename or path structure similar to the downloaded notebook:

In [3]:
%%bash

ls -l downloaded/ps1/archive

total ##
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] jupyter.png
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] notebooks.zip
-rw------- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb


Before we extract the files, we also need to have run ``nbgrader generate_assignment``:

In [4]:
%%bash

nbgrader generate_assignment "ps1" --IncludeHeaderFooter.header=source/header.ipynb --force

[GenerateAssignmentApp | WARNING] Removing existing assignment: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/release/ps1
[GenerateAssignmentApp | INFO] Copying [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/source/./ps1/jupyter.png -> [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/release/./ps1/jupyter.png
[GenerateAssignmentApp | INFO] Updating/creating assignment 'ps1': {}
[GenerateAssignmentApp | INFO] Converting notebook [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/source/./ps1/problem1.ipynb
[GenerateAssignmentApp | INFO] Writing [size] bytes to [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/release/ps1/problem1.ipynb
[GenerateAssignmentApp | INFO] Converting notebook [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/source/./ps1/problem2.ipynb
[GenerateAssignmentApp | INFO] Writing [size] bytes to [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/release/ps1/problem2.ipynb
[GenerateAssignmentApp | INFO] Setting destination file permissions to 644


### Step 2: Extract, collect, and copy files

In [5]:
%%bash

nbgrader zip_collect ps1 --force

[ZipCollectApp | INFO] Using file extractor: ExtractorPlugin
[ZipCollectApp | INFO] Using file collector: FileNameCollectorPlugin
[ZipCollectApp | WARNING] Directory not found. Creating: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted
[ZipCollectApp | INFO] Copying from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/jupyter.png
[ZipCollectApp | INFO]   Copying to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/jupyter.png
[ZipCollectApp | INFO] Extracting from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/notebooks.zip
[ZipCollectApp | INFO]   Extracting to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/notebooks
[ZipCollectApp | INFO] Copying from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb
[ZipCollectApp | INFO]   Copying to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_gu

After running the ``nbgrader zip_collect`` sub-command, the archive (zip) files were extracted - and the non-archive files were copied - to the ``extracted_directory``:

In [6]:
%%bash

ls -l downloaded/ps1/extracted/
ls -l downloaded/ps1/extracted/notebooks/

total ##
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] jupyter.png
drwxrwxr-x 1 nb_user nb_group [size] [date] [time] notebooks
-rw------- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb
total ##
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_bitdiddle_attempt_2016-01-30-15-30-10_jupyter.png
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_bitdiddle_attempt_2016-01-30-15-30-10_problem1.ipynb
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_bitdiddle_attempt_2016-01-30-15-30-10_problem2.ipynb
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-16-30-10_jupyter.png
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-16-30-10_myproblem1.ipynb
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] ps1_hacker_attempt_2016-01-30-16-30-10_problem2.ipynb


In [7]:
%%bash

ls -l submitted_zip

total ##
drwxrwxr-x 1 nb_user nb_group [size] [date] [time] bitdiddle
drwxrwxr-x 1 nb_user nb_group [size] [date] [time] hacker


In [8]:
%%bash

ls -l submitted_zip/hacker/ps1/

total ##
-rw------- 1 nb_user nb_group [size] [date] [time] problem1.ipynb
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] problem2.ipynb
-rw-rw-r-- 1 nb_user nb_group [size] [date] [time] timestamp.txt


## Custom plugins

In [9]:
%%bash

cat submitted_zip/hacker/ps1/timestamp.txt

2016-01-31 06:00:00

This is an issue with the underlying ``dateutils`` package used by ``nbgrader``. But not to worry, we can easily create a custom collector plugin to correct the timestamp strings when the files are collected, for example:

In [10]:
%%file plugin.py

from nbgrader.plugins import FileNameCollectorPlugin


class CustomPlugin(FileNameCollectorPlugin):
    def collect(self, submission_file):
        info = super(CustomPlugin, self).collect(submission_file)
        if info is not None:
            info['timestamp'] = '{}-{}-{} {}:{}:{}'.format(
                *tuple(info['timestamp'].split('-'))
            )
        return info

Writing plugin.py


In [11]:
%%bash

# Use force flag to overwrite existing files
nbgrader zip_collect --force --collector=plugin.CustomPlugin ps1

[ZipCollectApp | INFO] Using file extractor: ExtractorPlugin
[ZipCollectApp | INFO] Using file collector: CustomPlugin
[ZipCollectApp | WARNING] Clearing existing files in [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted
[ZipCollectApp | INFO] Copying from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/jupyter.png
[ZipCollectApp | INFO]   Copying to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/jupyter.png
[ZipCollectApp | INFO] Extracting from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/notebooks.zip
[ZipCollectApp | INFO]   Extracting to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/extracted/notebooks
[ZipCollectApp | INFO] Copying from: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/ps1/archive/ps1_hacker_attempt_2016-01-30-20-30-10_problem1.ipynb
[ZipCollectApp | INFO]   Copying to: [NB_GRADER_ROOT]/nbgrader/docs/source/user_guide/downloaded/

The ``--force`` flag is used this time to overwrite existing extracted and submitted files. Now if we check the timestamp we see it parsed correctly:

In [12]:
%%bash

cat submitted_zip/hacker/ps1/timestamp.txt

2016-01-30 20:30:10

Note that there should only ever be *one* instructor who runs the ``nbgrader zip_collect`` command (and there should probably only be one instructor -- the same instructor -- who runs `nbgrader generate_assignment`, `nbgrader autograde` and `nbgrader formgrade` as well). However this does not mean that only one instructor can do the grading, it just means that only one instructor manages the assignment files. Other instructors can still perform grading by accessing the formgrader URL.